In [1]:
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [287]:
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
import pandas as pd
import random
import numpy as np
from nltk.stem import PorterStemmer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Input, LSTM, Dropout
import math
from keras.models import Model
from tensorflow.keras.utils import to_categorical
import nltk
from sklearn.model_selection import train_test_split

In [271]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [311]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.stem import WordNetLemmatizer
  
lemmatizer = WordNetLemmatizer()
  
 
def clean_doc(article):
            
            #print('article before ',article)
            tokens =  [word for word in article.split()]
            table = str.maketrans('', '', punctuation)# remove punctuation from each token
            tokens = [w.translate(table) for w in tokens]
            tokens = [word for word in tokens if word.isalpha()]# remove remaining tokens that are not alphabetic
            stop_words = set(stopwords.words('english'))
            tokens = [w for w in tokens if not w in stop_words]# filter out stop words
            tokens = [lemmatizer.lemmatize(word) for word in tokens if len(word) > 1 and not '@' in word]# filter out short tokens
            #lemmatizer.lemmatize(word)
           # print('article after',TreebankWordDetokenizer().detokenize(tokens))
            return TreebankWordDetokenizer().detokenize(tokens)

In [312]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# New Section

In [313]:
!ls "/content/drive/MyDrive/bbc"
import os;
import sys;
from google.colab import drive
TEXT_DATA_DIR = '/content/drive/MyDrive/bbc'
texts = [] # list of text samples
labels = [] # list of label ids
labels_name=[]
index=0
for name in os.listdir(TEXT_DATA_DIR):
  if '.ipynb_checkpoints'==name:
    continue
  labels_name.append(name)
  path = os.path.join(TEXT_DATA_DIR, name)
  for fname in os.listdir(path):
    fpath = os.path.join(path, fname)
    args = {} if sys.version_info < (3,) else {'encoding': 'latin-1'}
    with open(fpath, **args) as f:
       t = f.read()
       texts.append(clean_doc(t))
    labels.append(index) 
  index+=1

            #print(fname)
                
#print(texts)
#print('nLabels = ', len(labels))
#print('Classes are:\n ')
#for key in labels:
 #   print (key)

        

business  politics  sport


In [314]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index # the dictionary 
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)


In [315]:
labels_matrix = to_categorical(np.asarray(labels))



In [316]:
xtrain,xtest,ytrain,ytest= train_test_split(data, labels_matrix,test_size=0.2,random_state=1)

In [317]:
EMBEDDING_DIM = 100
print('Indexing word vectors.')
embeddings_index = {}
with open('glove.6B.100d.txt') as f:
 for line in f:
   values = line.split(sep=' ')
   word = values[0]
   coefs = np.asarray(values[1:], dtype='float32')
   embeddings_index[word] = coefs
#for key, value in embeddings_index.items():
 #   print(key, ' : ', value)   
#print('Found word vectors.' ,(embeddings_index.most_common()))
 

Indexing word vectors.


In [318]:
# (7) Map the dataset dictionary of (words,IDs) to a matrix of the
# embeddings of each word in the dictionary
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))#+1 to include the zeros vector for non-existing words
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
print ('Shape of Embedding Matrix: ',embedding_matrix.shape)

Shape of Embedding Matrix:  (21005, 100)


In [319]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,  EMBEDDING_DIM, 
weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH,))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.summary()



Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_22 (Embedding)    (None, 1000, 100)         2100500   
                                                                 
 lstm_10 (LSTM)              (None, 128)               117248    
                                                                 
 dense_28 (Dense)            (None, 3)                 387       
                                                                 
Total params: 2,218,135
Trainable params: 2,218,135
Non-trainable params: 0
_________________________________________________________________


In [320]:

model.compile(loss='categorical_crossentropy',
 optimizer='rmsprop',
 metrics=['acc'])
# happy learning!
print(xtrain.shape,ytrain.shape,xtest.shape,ytest.shape)
model.fit(xtrain, ytrain, validation_data=(xtest, ytest),
 epochs=5, batch_size=128)

(1150, 1000) (1150, 3) (288, 1000) (288, 3)
Epoch 1/5
9/9 [==============================] - 57s 6s/step - loss: 0.8062 - acc: 0.6661 - val_loss: 0.3350 - val_acc: 0.9201
Epoch 2/5
9/9 [==============================] - 53s 6s/step - loss: 0.2826 - acc: 0.9122 - val_loss: 0.2006 - val_acc: 0.9306
Epoch 3/5
9/9 [==============================] - 53s 6s/step - loss: 0.1725 - acc: 0.9461 - val_loss: 1.1329 - val_acc: 0.6944
Epoch 4/5
9/9 [==============================] - 53s 6s/step - loss: 0.3025 - acc: 0.9061 - val_loss: 0.1331 - val_acc: 0.9583
Epoch 5/5
9/9 [==============================] - 55s 6s/step - loss: 0.1093 - acc: 0.9678 - val_loss: 0.1036 - val_acc: 0.9688


In [321]:
print('Acuracy on testing set:')
model.evaluate(xtest,ytest)

Acuracy on testing set:
9/9 [==============================] - 4s 318ms/step - loss: 0.1036 - acc: 0.9688


[0.10358762741088867, 0.96875]

In [322]:
def predict(test):
    cleanedTest=clean_doc(test)
    #print("cleanedTestResult",cleanedTestResult)
    tokenizer.fit_on_texts([cleanedTest])
    encoded_test= tokenizer.texts_to_sequences([cleanedTest])
    #print("encoded_test",encoded_test)
    # pad sequencesmax_length
    Xtest = pad_sequences(encoded_test, maxlen=MAX_SEQUENCE_LENGTH)
    #print("xtest",Xtest)
    res=model.predict([Xtest])
    label_id = np.argmax(res)
    print(res)
    for  ID in labels:
        if label_id == ID:
             print ('The category of article %s is %s' %(test,labels_name[ID]))
             break

In [323]:
# batch size =64 and applying lemmitization th accuracy =96 in training and 98 in testing
#batch size =64 and without lemmitization th accuracy =93 in training and 96 in testing
# batch size =128 and applying lemmitization th accuracy =95 in training and 96 in testing

predict('i love football')
predict('we expect higher revenue and wider profit because i want to expand my business')
predict('he was a Democrat in america')

[[0.5388224  0.21855034 0.24262723]]
The category of article i love football is sport
[[0.05708252 0.65434134 0.28857616]]
The category of article we expect higher revenue and wider profit because i want to expand my business is business
[[0.33016875 0.29443458 0.37539667]]
The category of article he was a Democrat in america is politics


# New Section